In [2]:
import os

os.chdir('C:/Users/AElegido/Downloads/Routes')
print(os.getcwd())

C:\Users\AElegido\Downloads\Routes


<h1><center><b><font size="5">ROUTE ALARMS</font></b></center></h1>

In [3]:
import pandas as pd
import datetime

# Load the CSV file
df = pd.read_csv('Alarms_2023-05-02_2023-05-02.csv')

# convert Time Column to datetime format
df['Time'] = pd.to_datetime(df['Time'])

# Filter the Time column based on the specified time ranges
time_range_1 = df[(df['Time'].dt.time >= datetime.time(17, 40)) & (df['Time'].dt.time <= datetime.time(18, 20))]
time_range_2 = df[(df['Time'].dt.time >= datetime.time(20, 40)) & (df['Time'].dt.time <= datetime.time(21, 20))]


# TimeRange 3
start_time = pd.Timestamp('2023-03-14 23:40:00')
end_time = pd.Timestamp('2023-03-15 00:20:00')
time_range_3 = df[(df['Time'] >= start_time) & (df['Time'] <= end_time)]

time_range_4 = df[(df['Time'].dt.time >= datetime.time(2, 40)) & (df['Time'].dt.time <= datetime.time(3, 20))]
time_range_5 = df[(df['Time'].dt.time >= datetime.time(5, 40)) & (df['Time'].dt.time <= datetime.time(6, 20))]
time_range_6 = df[(df['Time'].dt.time >= datetime.time(8, 40)) & (df['Time'].dt.time <= datetime.time(9, 20))]

# Remove unwanted columns
columns_to_remove = ['From', 'To', 'Voyage Code', 'Note Time', 'Note Title', 'Note Text', 'Name of User']
time_range_1 = time_range_1.drop(columns=columns_to_remove)
time_range_2 = time_range_2.drop(columns=columns_to_remove)
time_range_3 = time_range_3.drop(columns=columns_to_remove)
time_range_4 = time_range_4.drop(columns=columns_to_remove)
time_range_5 = time_range_5.drop(columns=columns_to_remove)
time_range_6 = time_range_6.drop(columns=columns_to_remove)

# Remove duplicates in Vessel Column
time_range_1.drop_duplicates(subset=['Vessel', 'Route Alarm', 'Stoppage Alarm'], inplace=True)
time_range_2.drop_duplicates(subset=['Vessel', 'Route Alarm', 'Stoppage Alarm'], inplace=True)
time_range_3.drop_duplicates(subset=['Vessel', 'Route Alarm', 'Stoppage Alarm'], inplace=True)
time_range_4.drop_duplicates(subset=['Vessel', 'Route Alarm', 'Stoppage Alarm'], inplace=True)
time_range_5.drop_duplicates(subset=['Vessel', 'Route Alarm', 'Stoppage Alarm'], inplace=True)
time_range_6.drop_duplicates(subset=['Vessel', 'Route Alarm', 'Stoppage Alarm'], inplace=True)


# Save the filtered dataframes to separate CSV files
time_range_1.to_csv('alarms1800.csv', index=False)
time_range_2.to_csv('alarms2100.csv', index=False)
time_range_3.to_csv('alarms2400.csv', index=False)
time_range_4.to_csv('alarms0300.csv', index=False)
time_range_5.to_csv('alarms0600.csv', index=False)
time_range_6.to_csv('alarms0900.csv', index=False)

<h1><center><b><font size="5">REPORTED WEATHER REPORT EVERY 0200 UTC</font></b></center></h1>

In [4]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill
from openpyxl.styles import Alignment

# Load CSV file into a Pandas DataFrame
df = pd.read_csv('noon-reports-202305020908.csv')

# Select only the required columns
df = df[['Vessel Name', 'Report Type', 'Message date (LT)', 'Message date (GMT)', 'Condition', 'Speed (knots)', 'Voyage ordered speed (knots)', 'Voyage avg speed (knots)', 'Wind Direction (Degree)', 'Wind force (Bft)', 'Current Direction', 'Current Speed (knots)', 'Sea direction (Degree)', 'Sea state (Douglas scale)', 'Arrival port', 'Departure port', 'Next port ETA (GMT)', 'Next port ETA (LT)']]

# Convert Wind force (Bft) to Integer
df['Wind force (Bft)'] = df['Wind force (Bft)'].fillna(0).astype(int)

# Filter to Wind Force greater or equal to 7
df = df[df['Wind force (Bft)'] >= 7]


# Save the filtered DataFrame as an Excel file
workbook = Workbook()
worksheet = workbook.active

# Write the DataFrame to the worksheet
for row in dataframe_to_rows(df, index=False, header=True):
    worksheet.append(row)

# Format the Wind force (Bft) column
wind_force_col = worksheet['J']
for cell in wind_force_col[1:]:
    if cell.value >= 7:
        cell.font = Font(bold=True, color='FF0000')  # Set bold and red font
        
# Auto-fit the column width
for column_cells in worksheet.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    worksheet.column_dimensions[column_cells[0].column_letter].width = length
    
# Set row height to 20
for row in worksheet.rows:
    row_height = 20
    for cell in row:
        cell.alignment = Alignment(horizontal='center', vertical='center')
        # Set horizontal and vertical center alignment
    worksheet.row_dimensions[row[0].row].height = row_height

# Save the updated Excel file
workbook.save('noon-reports-202305020908.xlsx')

<h1><center><b><font size="5">DOWNLOADING EXCEL ROUTE PLAN THEN CONVERTING TO CSV AND RTZ</font></b></center></h1>

In [1]:
import os
import re
import pandas as pd
import win32com.client
from datetime import datetime, timezone, timedelta

# Set up connection to Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.Folders["ccrops@csmcy.com"].Folders["Inbox"]

# Define the folder where you want to save the files
output_folder = r"C:/Users/AElegido/Downloads/Routes"

# Define the date and time from which you want to save attachments
cyprus_time_str = input("Enter the date and time from which you want to save attachments in the format 'YYYY-MM-DD HH:MM:SS': ")
cyprus_time = datetime.strptime(cyprus_time_str, "%Y-%m-%d %H:%M:%S")
utc_time = cyprus_time - timedelta(hours=2) # Cyprus Time is 2 hours ahead of UTC
utc_timezone = timezone.utc

# Loop through the emails in your Inbox
for message in inbox.Items:
    try:
        # Make the CreationTime attribute timezone-aware
        creation_time = message.CreationTime.replace(tzinfo=timezone(timedelta(hours=2)))

        # Check if the email has been created after the specified date and time
        if creation_time > utc_time.replace(tzinfo=utc_timezone):
            # Check if the email has an attachment
            if message.Attachments.Count > 0:
                # Loop through the attachments
                for attachment in message.Attachments:
                    # Check if the attachment is an Excel or RTZ file
                    if re.search(r"\.xls[xm]?$", attachment.FileName, re.IGNORECASE) or re.search(r"\.rtz$", attachment.FileName, re.IGNORECASE):
                        # Save the file to the output folder
                        attachment.SaveAsFile(os.path.join(output_folder, attachment.FileName))

                        # If the file is an Excel file, read it as a Pandas dataframe and save it as a CSV file
                        if re.search(r"\.xls[xm]?$", attachment.FileName, re.IGNORECASE):
                            excel_file = pd.read_excel(os.path.join(output_folder, attachment.FileName))
                            csv_file_name = os.path.splitext(attachment.FileName)[0] + ".csv"
                            excel_file.to_csv(os.path.join(output_folder, csv_file_name), index=False)
    except Exception as e:
        print(f"Error processing email: {e}")

Enter the date and time from which you want to save attachments in the format 'YYYY-MM-DD HH:MM:SS': 2023-06-24 20:27:00
